In [2]:
# Library imports

# core just in case
import os
import sys

# Data handling
import numpy as np
import pandas as pd

# ML framework (TensorFlow -> includes Keras)
import torch
from torch.utils.data import Dataset

# Hugging Face * had a small error with transformers -> (pip install transformers datasets huggingface_hub) -> error go byebye.
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
from huggingface_hub import notebook_login

# Evaluation / Utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Logging / visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Note had to install earlier version of Keras via (pip install tf-keras) -> restart kernel -> working:)


In [9]:
# Load Dataset
df = pd.read_csv("./data/sentiment.csv")  # replace with actual path
df = df.dropna(subset=["statement", "status"])  # safety first

In [10]:
# Encode labels
le = LabelEncoder()
df["label"] = le.fit_transform(df["status"])
num_labels = len(le.classes_)

In [11]:
# Split
df_filtered = df[df['label'] != 1].copy() # Remove the row with label 1
train_df, val_df = train_test_split(df_filtered, test_size=0.1, stratify=df_filtered["label"], random_state=42)

In [12]:
# BERT stuff
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

class MentalHealthDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MentalHealthDataset(train_df["statement"].tolist(), train_df["label"].tolist(), tokenizer)
val_dataset   = MentalHealthDataset(val_df["statement"].tolist(), val_df["label"].tolist(), tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
# Model Train & Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    # Get unique labels present in the current evaluation batch
    unique_labels = np.unique(labels)
    # Filter le.classes_ to include only present labels
    present_target_names = le.classes_[unique_labels]
    report = classification_report(labels, preds, target_names=present_target_names, labels=unique_labels, output_dict=True)
    macro_recall = report["macro avg"]["recall"]
    print("Macro Recall:", macro_recall)
    return {"macro_recall": macro_recall}

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Training Args
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels
)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",       # or evaluation_strategy in newer versions
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="macro_recall",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Macro Recall
1,0.478100,0.408736,0.776596
2,0.355000,0.405241,0.801127
3,0.214300,0.488375,0.821673


Macro Recall: 0.776595811012314
Macro Recall: 0.8011267662246238
Macro Recall: 0.8216732550475964


TrainOutput(global_step=8424, training_loss=0.35062376590196226, metrics={'train_runtime': 3377.0901, 'train_samples_per_second': 39.898, 'train_steps_per_second': 2.494, 'total_flos': 8863228000915200.0, 'train_loss': 0.35062376590196226, 'epoch': 3.0})

In [18]:
# Eval
preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = val_df["label"].values

# Get unique labels present in y_true
unique_labels_true = np.unique(y_true)
# Filter le.classes_ to include only present labels in y_true
present_target_names_true = le.classes_[unique_labels_true]

print(classification_report(y_true, y_pred, target_names=present_target_names_true, labels=unique_labels_true))

Macro Recall: 0.8216732550475964
                      precision    recall  f1-score   support

             Anxiety       0.90      0.92      0.91       384
          Depression       0.81      0.77      0.79      1541
              Normal       0.96      0.96      0.96      1634
Personality disorder       0.78      0.76      0.77       108
              Stress       0.76      0.75      0.76       259
            Suicidal       0.72      0.76      0.74      1065

            accuracy                           0.84      4991
           macro avg       0.82      0.82      0.82      4991
        weighted avg       0.84      0.84      0.84      4991

